## Download the Module first

In [1]:
pip install alphai-api

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from alphai import *

Connected to AlphAI, type help(alphai) for more info


### Functions we are using in this task
* motor(left,right,time): used to control the moveing of the robots
* set_distance(True/False): open up the ultrasound sensor
* get_distance(): return the distance to obstacles(in cm)
* set_camera(str/none): str could be "2x1","4x3"...,"64x68" that means the pixels. If you input nothing, it will close down the camera
* get_camera(): return a integer list of the image
* is_connected(): return a boolean value about whether the robot has been conneted

## Sart Navigation and programing
* Navigation with ultra sound
* Navigation with camera

In [3]:
# Connect to robot's Wi-Fi first and test the connection
connect_wifi()
if is_connected():
    print("Connected to the robot successfully!")
# is_connected: a boolean value indicating whether the connection is successful 

# Prepare remote control
def forward():
    motor(15, 15, 0.3)  # Move forward for 1 seconds

def backward():
    motor(-15, -15, 0.3)  # Move backward for 1 seconds

def left():
    motor(-20, 20, 0.1)  # Turn left for 1 seconds

def right():
    motor(20, -20, 0.1)  # Turn right for 1 seconds
    
def stop():
    motor(0, 0, 1)  # Stop the robot for 1 second

attempt connection wifi
Connection to robot successful
Connected to the robot successfully!


Made changes.

The code now collects ultrasound data for 20 seconds, stores each reading with a timestamp in a pandas DataFrame, and prints the DataFrame with the real time as the leftmost column. You can run the updated cell to see the results.

## Navigation with ultra sound
In ths task we will use following functions to create a autonomous function ultrasound_race() that could let the robot to make decisions based on the distances to obstacle.
* get_distance() --> returns the distance measured by the robot's ultrasound sensor
* set_distance(True) --> activate the ultrasound sensor
* <span style="color:red">Careful that if the robot is placed in front of a wall with an angle, the distance will be greater since the ultrasound will reflect off the wall
* In the following code, we created a dataframe to store all values reported from robots which should be sensed every two seconds.

In [4]:
import pandas as pd
from datetime import datetime
import time

# Set up the ultrasound sensor
set_distance(True) # activate the ultrasound sensor

# Collect ultrasound data once per second for 20 seconds
data = []
for i in range(20):
    d = get_distance() # get the distance measured by the sensor
    timestamp = datetime.now().strftime('%Y/%m/%d %H:%M:%S')
    data.append([timestamp, d])
    #print(f"{timestamp} distance: {d}")
    time.sleep(2)  # wait for 2 second

# Create DataFrame with timestamp as leftmost column
df = pd.DataFrame(data, columns=["Time", "Distance (cm)"])
print(df)

                   Time  Distance (cm)
0   2025/08/06 14:04:02      19.892395
1   2025/08/06 14:04:04      18.616665
2   2025/08/06 14:04:06      19.818794
3   2025/08/06 14:04:08       8.427179
4   2025/08/06 14:04:10      10.667885
5   2025/08/06 14:04:12       5.209231
6   2025/08/06 14:04:15       5.250120
7   2025/08/06 14:04:17       9.265399
8   2025/08/06 14:04:19      12.238014
9   2025/08/06 14:04:21      16.997469
10  2025/08/06 14:04:23      16.972935
11  2025/08/06 14:04:25      17.038357
12  2025/08/06 14:04:27      16.968845
13  2025/08/06 14:04:29      16.911602
14  2025/08/06 14:04:31      16.968845
15  2025/08/06 14:04:33      16.932047
16  2025/08/06 14:04:35      16.952492
17  2025/08/06 14:04:37      17.042446
18  2025/08/06 14:04:39      16.993380
19  2025/08/06 14:04:41      17.026091


Next, we are trying to define an automatical function: ultrasound_race to help us away from the obstacle.
* <span style="color:red">Here we ask robot to go forward if there is no obstacle (distance >10) and ask it to turn right when it detects an obstacle (distance<10)

In [4]:
def forward():
    motor(15, 15, 0.1) # move forward at speed 15 for 0.1 seconds
def right():
    motor(20, -20, 0.1) # turn right at speed 20 for 0.1 seconds
def ultrasound_race():
    d = get_distance()
    if d < 10:  # if the distance is less than 10 cm
        print("Obstacle detected! Turn right.")
        right()  # turn right
    else:
        print("Path clear. Move forward.")
        forward()

Last, we test our program in the main function

In [ ]:
if __name__ == '__main__':
    # Test the ultrasound
    
    set_distance(True) # activate the ultrasound sensor

    ultrasound_race()

    set_distance(False) # deactivate the ultrasound sensor

## Nevigation with the camera
The camera will film an image with a grid of pixels. In a color image, each pixel contains 3 values: red, green and blue. In this task, we are trying to complete camera_race() function to let the robot to make decisions based on camera sensors.
Following are the things we may need:
* set_camera("8x6") --> set the mode of camera
* get_camera() --> This function will return a matrix of list indicating the different values for each pixel including three colors **(If the color is dark that means the obstacle is close, the values will be lower --> usually we use 100 as seperation)**
* <span style="color:red">In other words, we could access the pixel value of row i, column j and in channel k (k = 0 for red, k = 1 for green, 2 for blue) with image[i][j][k] where image = get_camera()</span>
* For example, in the folowing codes, we are asking it to return us the values of 3 channals of the first row with the second column as circled in the figure

![Sample Camera image](8x6%20Camera.png)

In [19]:
set_camera("8x6") # set the camera to the correct resolution
for i in range(10):
    image = get_camera() # get the camera image
    print('red', image[0][1][0], 'green', image[0][1][1], 'blue', image[0][1][2])
# red 205 green 204 blue 207

red 148 green 131 blue 137
red 150 green 133 blue 136
red 134 green 129 blue 127
red 150 green 129 blue 133
red 168 green 139 blue 145
red 169 green 141 blue 149
red 164 green 138 blue 145
red 164 green 137 blue 143
red 169 green 142 blue 148
red 173 green 140 blue 148


### Next we create a dataframe to store 20 data with a 2 second time interval

In [10]:
import pandas as pd
from datetime import datetime
import time

set_camera("8x6") # set the camera to the correct resolution

# Collect camera data once per second for 20 seconds
data = []
for i in range(20):
    image = get_camera() # get the camera image
    timestamp = datetime.now().strftime('%Y/%m/%d %H:%M:%S')
    data.append([timestamp, image])
    time.sleep(2)  # wait for 2 second

# Create DataFrame with timestamp as leftmost column
df = pd.DataFrame(data, columns=["Time", "Image Data"])
print(df)

                   Time                                         Image Data
0   2025/08/01 15:09:28  [[[94, 121, 127], [142, 153, 144], [152, 162, ...
1   2025/08/01 15:09:30  [[[95, 121, 127], [142, 152, 143], [152, 161, ...
2   2025/08/01 15:09:33  [[[95, 121, 127], [142, 153, 142], [152, 162, ...
3   2025/08/01 15:09:35  [[[94, 121, 126], [142, 154, 142], [152, 163, ...
4   2025/08/01 15:09:37  [[[94, 121, 129], [143, 154, 146], [153, 164, ...
5   2025/08/01 15:09:39  [[[96, 120, 127], [144, 153, 143], [154, 162, ...
6   2025/08/01 15:09:41  [[[95, 121, 128], [143, 153, 143], [153, 161, ...
7   2025/08/01 15:09:43  [[[97, 121, 125], [146, 155, 144], [155, 165, ...
8   2025/08/01 15:09:45  [[[94, 120, 126], [143, 153, 142], [153, 163, ...
9   2025/08/01 15:09:48  [[[93, 120, 126], [141, 152, 141], [152, 161, ...
10  2025/08/01 15:09:50  [[[95, 119, 127], [143, 152, 143], [153, 162, ...
11  2025/08/01 15:09:52  [[[94, 120, 124], [142, 152, 141], [152, 161, ...
12  2025/08/01 15:09:54  

Next, we are trying to define an automatical function called camera_race() which based only on the color of a piexel in the bottom part of the image (like image[4][3][0/1/2])

In [ ]:
def camera_race():
    image = get_camera()
    # Check region for right: rows 2,3,4; columns 0,1,2,3,4,5
    right_region = [image[r][c] for r in [2,3,4] for c in [0,1,2,3]]
    # Check region for left: rows 2,3,4; columns 2,3,4,5,6,7
    left_region = [image[r][c] for r in [2,3,4] for c in [4,5,6,7]]
    # Check region for backward: rows 2,3,4; columns 0,1,2
    backward_region = [image[r][c] for r in [2,3,4] for c in [2,3,4,5]]

    # Helper to check if at least 30% of pixels in region are "dark" (all channels < 100)
    def is_most_dark(region, threshold=0.4):
        dark_count = sum(
            sum(channel < 90 for channel in pixel) for pixel in region
        )
        total_channels = len(region) * len(region[0]) if region else 1
        return dark_count / total_channels > threshold
    if is_most_dark(backward_region, threshold=0.4):
        print("No clear path, gobackward")
        backward()
    elif is_most_dark(right_region, threshold=0.3):
        print("Obstacle on left, turn right")
        right()
    elif is_most_dark(left_region, threshold=0.3):
        print("Obstacle on right, turn left")
        left()
    else:
        print("Path clear, go forward")
        forward()

Last, let's test in the main function

In [11]:
if __name__ == '__main__':
    # Test
    set_camera("8x6")

    for i in range(30):
        camera_race()

    set_camera() # deactivate the camera sensor

Path clear, go forward
Path clear, go forward
Obstacle on left, turn right
Obstacle on right, turn left
Path clear, go forward
Obstacle on left, turn right
Obstacle on left, turn right
Path clear, go forward
Obstacle on right, turn left
Obstacle on left, turn right
Path clear, go forward
Path clear, go forward
Obstacle on left, turn right
Path clear, go forward
Obstacle on right, turn left
Obstacle on right, turn left
Obstacle on right, turn left
Obstacle on left, turn right
Obstacle on left, turn right
Obstacle on left, turn right
Obstacle on left, turn right
Obstacle on right, turn left
Obstacle on left, turn right
Path clear, go forward
Obstacle on left, turn right
Path clear, go forward
Obstacle on right, turn left
Obstacle on right, turn left
Obstacle on left, turn right
Obstacle on left, turn right
